## Script to click on the Show All option and extract data

In [ ]:
# Should have chrome installed on the machine running
# Install selenium
#!pip install selenium
# Install yahooquery
#!pip install yahooquery
# ! Optional - download chrome driver based on OS and keep it in the same folder which has this notebook

In [1]:
import re
import platform
from urllib.request import urlopen
from zipfile import ZipFile
from selenium import webdriver
import time 
import pandas as pd
import os
from datetime import datetime
from yahooquery import Ticker

In [10]:
# Download chromedriver -- !! Latest One which matches with Chrome browser !!
plat = platform.system()
print(plat)
if re.search('Windows', plat):
    zipurl = "https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip"

###### Not tested on Linux yet
elif re.search('Linux', plat):
    zipurl = "https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip"

elif re.search('Darwin', plat):
    zipurl = "https://chromedriver.storage.googleapis.com/107.0.5304.62/chromedriver_mac64.zip"
    #"https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_mac64.zip"
    #"https://chromedriver.storage.googleapis.com/index.html?path=107.0.5304.62/"
    
zipresp = urlopen(zipurl)
tempzip = open("tempfile.zip", "wb")
tempzip.write(zipresp.read())
tempzip.close()
zf = ZipFile("tempfile.zip", "r")
try: # If chromedriver is already downloaded, the next line throws errors but the script works. Hence the pass statement.
    zf.extractall()
except Exception as e:
    pass
zf.close()

Darwin


In [11]:
!ls

SPAC_Extraction_Automation (1).ipynb tempfile.zip
chromedriver


In [12]:
if plat=='Darwin':
    os.chmod('./chromedriver', 755) # e.g. os.chmod('/Users/user/Documents/my_project/chromedriver', 0755)


In [13]:
# Get data
driver = webdriver.Chrome(os.getcwd()+'/chromedriver') # Need complete absolute path here
driver.get("https://www.defianceetfs.com/spacresearch") #put here the adress of your page

elem = driver.find_elements_by_xpath("//*[@id='filters-dropdown-lenght']") #outter dropdown button
elem2 = driver.find_elements_by_xpath("//*[@id='table-research_length']/label/div[2]/div/button[4]") #Show All button

elem[0].click()
time.sleep(5)

elem2[0].click()
item = driver.find_elements_by_xpath("//*[@id='table-research']//a")
time.sleep(3)

counter = 0
for table in driver.find_elements_by_xpath("//*[@id='table-research']//tr"):
    data = [item.text for item in table.find_elements_by_xpath(".//*[self::td or self::th]")]
    if len(data) > 9:
        if counter == 0:
            data_df = pd.DataFrame(columns=data)
            counter+=1
        else:
            df_1 = pd.DataFrame([data],columns=data_df.columns)
            data_df = data_df.append(df_1, ignore_index=True)
            
driver.quit()
print(data_df.shape)
data_df.head()

IndexError: list index out of range

In [ ]:
def convertDate(date='07/14/2020'):
    datList=date.split('/')
    if len(datList) > 1:
        date=datList[2]+'-'+datList[0]+'-'+datList[1]
    return date

In [ ]:
data_df["IPO date"] = data_df["IPO date"].apply(lambda x: convertDate(x)) 
data_df["Liquidation date"] = data_df["Liquidation date"].apply(lambda x: convertDate(x)) 
data_df["Trust acc/share"] = data_df["Trust acc/share"].apply(lambda x: re.sub("[^0-9^.]", "", x)) 
data_df["Trust account"] = data_df["Trust account"].apply(lambda x: re.sub("[^0-9^.]", "", x)) 
ts = datetime.fromtimestamp(datetime.timestamp(datetime.now()))
data_df["TS"]=ts
data_df

In [ ]:
def getInfo(ticker):
    
    try:
        df=pd.DataFrame(columns=["Name", "Type", "Percentage"])
        ticker=ticker.upper()
        company = Ticker(ticker)
        try:
            offcnameList=list(company.company_officers.name)
            offctitleList=list(company.company_officers.title)
            df_ofc = pd.DataFrame({'Name':offcnameList,'Type':offctitleList})
            df_ofc["Percentage"]=0.0
            for index, row in df_ofc.iterrows():
                cleaned_name = re.sub(r'(^\w{2,3}\. ?)', r'', row["Name"])
                df_ofc.at[index, 'Name']=cleaned_name.lower()
            df=df.append(df_ofc)
        except Exception as e:
            print("1. company Leadership", ticker,str(e))
        
        
        try:
            df_ins = pd.DataFrame({'Name':list(company.institution_ownership.organization),'Percentage':list(company.institution_ownership.pctHeld)})
            df_ins["Type"]="Institutuional Holder"
            df=df.append(df_ins)
        except Exception as e:
            print("2. Institutional holder", ticker,str(e))     
        
        try:
            df_mf = pd.DataFrame({'Name':list(company.fund_ownership.organization),'Percentage':list(company.fund_ownership.pctHeld)})
            df_mf["Type"]="Mutual Fund Holder"
            df=df.append(df_mf)
        except Exception as e:
            print("3. MututalFund holder", ticker,str(e)) 
            
        
        if df.shape[0] > 0:
            df["Symbol"]=ticker
            
        return df
    except Exception as e:
        print(ticker, str(e))

In [ ]:
df_holders=pd.DataFrame(columns=["Name", "Type", "Percentage"])
for index, row in data_df.iterrows():
    df2=getInfo(row['Symbol'])
    try:
        if df2.shape[0]>0:
            df_holders=df_holders.append(df2)
    except Exception as e:
        pass
        #print(row['Ticker'], str(e))
df_holders["TS"]=ts

In [ ]:
df_holders.sort_values(by='Percentage', ascending=False)
df_holders

## Create csv files

In [14]:
#data_df.to_csv("SPAC_Data.csv", index=False)
#df_holders.to_csv("SPAC_holders.csv", index=False)
driver.quit()